In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import log_loss

#### read in data
+ read csvs
+ convert to matrices and get index

In [2]:
X_train = pd.read_csv("data/model_inputs/X_train.csv")

In [3]:
X_test = pd.read_csv("data/model_inputs/X_test.csv")

In [4]:
y_train = pd.read_csv("data/model_inputs/y_train.csv")

In [5]:
y_train_index = y_train.VisitNumber.values
y_train_dummies = pd.get_dummies(y_train.drop("VisitNumber",axis=1).TripType).as_matrix()
X_train = X_train.drop(['VisitNumber', 'id'], axis=1).as_matrix()
X_test = X_test.drop(['VisitNumber', 'id'], axis=1).as_matrix()

In [6]:
X_train.shape

(95674, 177)

#### splitting of train data set to train_ and validation set

In [7]:
sss = StratifiedShuffleSplit(y_train_dummies,1, test_size=0.25, random_state=0)
for train_index, test_index in sss:
    X_train_, X_val = X_train[train_index], X_train[test_index]
    y_train_, y_val = y_train_dummies[train_index], y_train_dummies[test_index]

In [8]:
print(X_train_.shape)
print(y_train_.shape)
print(X_val.shape)
print(y_val.shape)

(2726690, 177)
(2726690, 38)
(908922, 177)
(908922, 38)


#### data set function

In [9]:
# class for mini batches with the next_batch function
# also gives information about how much of data set is gone through

class DataSet(object):
    def __init__(self, predictors, labels):
        self._num_examples = predictors.shape[0]
        self._predictors = predictors
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
        
    @property
    def predictors(self):
        return self._predictors
    
    @property
    def labels(self):
        return self._labels
    
    @property
    def num_examples(self):
        return self._num_examples
    
    @property
    def epochs_completed(self):
        return self._epochs_completed
    

    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._predictors = self._predictors[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._predictors[start:end], self._labels[start:end]

#### tensorflow model

In [10]:
# model specification

#
# mlp 
#

# Parameters

#Network Parameters
n_hidden_1 = 128
n_hidden_2 = 128

n_input = X_train_.shape[1]  # total number of input variables
n_classes = y_train_.shape[1] # total number of classes

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# drop out
keep_prob_l1 = tf.placeholder("float")
keep_prob_l2 = tf.placeholder("float")



def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.tanh(tf.matmul(_X, _weights['h1']) + _biases['b1']) 
    layer_1_drop = tf.nn.dropout(layer_1, keep_prob_l1)
    layer_2 = tf.nn.tanh(tf.matmul(layer_1_drop, _weights['h2']) + _biases['b2'])
    layer_2_drop = tf.nn.dropout(layer_2, keep_prob_l2)
    return tf.matmul(layer_2_drop, weights['out']) + biases['out']

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

py_x = multilayer_perceptron(X, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y)) # compute mean cross entropy (softmax is applied internally)
train_op = tf.train.RMSPropOptimizer(learning_rate=0.001, decay=0.8).minimize(cost) # construct optimizer
model_output = tf.nn.softmax(py_x)

data_sets = DataSet(X_train_, y_train_)

#### train model

In [11]:
sess = tf.InteractiveSession()
tf.initialize_all_variables().run()

In [12]:
# run ops / training steps here
batch_size = 200


for i in range(100000):
    batch = data_sets.next_batch(batch_size)
    if i%5000 == 0:
        train_accuracy = cost.eval(feed_dict={X: batch[0], 
                                              Y: batch[1], 
                                              keep_prob_l1: 1.0, 
                                              keep_prob_l2: 1.0})
        print("step %d, softmax cross entropy %g"%(i, train_accuracy))
    train_op.run(feed_dict={X: batch[0], 
                            Y: batch[1], 
                            keep_prob_l1: 0.8, 
                            keep_prob_l2:0.8})
    
       
y_test_pred = sess.run(model_output,feed_dict={X: X_val, 
                                               keep_prob_l1: 1.0, 
                                               keep_prob_l2: 1.0})
log_loss(y_true=y_val, y_pred=y_test_pred)

step 0, softmax cross entropy 21.0316
step 5000, softmax cross entropy 1.80816
step 10000, softmax cross entropy 0.966205
step 15000, softmax cross entropy 1.11696
step 20000, softmax cross entropy 0.95208
step 25000, softmax cross entropy 0.83262
step 30000, softmax cross entropy 0.843167
step 35000, softmax cross entropy 1.0545
step 40000, softmax cross entropy 0.843817
step 45000, softmax cross entropy 0.799078
step 50000, softmax cross entropy 0.852599
step 55000, softmax cross entropy 0.827224
step 60000, softmax cross entropy 0.759751
step 65000, softmax cross entropy 0.695444
step 70000, softmax cross entropy 0.79962
step 75000, softmax cross entropy 0.718751
step 80000, softmax cross entropy 0.795504
step 85000, softmax cross entropy 0.703262
step 90000, softmax cross entropy 0.839027
step 95000, softmax cross entropy 0.67068


0.73592794632339409

In [14]:
sess.close()